In [2]:
import requests
import json
import pandas as pd
import re
import difflib
import numpy as np

# HDB Resale Data Feature Engineering with Geographic Data
This script enriches the HDB resale dataset with geographic coordinates and MRT proximity features.

In [3]:
hdb_resale_data = pd.read_csv("data/processed/hdb_resale_data.csv")

### 1. Fetch Latitude & Longitude from OneMap API for Each HDB unit

In [4]:
# ------------------------------------------------------------------------------
# This code used to fetch latitude and longitude using OneMap API
# Took approximately 5 hours 11 minutes to fetch 249,857 data
# ------------------------------------------------------------------------------

'''
hdb_resale_data['latitude'] = None
hdb_resale_data['longitude'] = None

for i, row in hdb_resale_data.iterrows():
    if pd.notna(row['latitude']) and pd.notna(row['longitude']):
        continue

    search_val = row['block'] + " " + row['street_name']
    url = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={search_val}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
    response = requests.get(url)

    if response.status_code == 200:
        best_score = 0
        best_latitude = 0
        best_longitude = 0

        for result in response.json()['results']:
            score = difflib.SequenceMatcher(None, result['ADDRESS'], search_val).ratio()
            if  score > best_score:
                best_score = score
                best_latitude = result['LATITUDE']
                best_longitude = result['LONGITUDE']
        
        hdb_resale_data.loc[i, 'latitude'] = best_latitude
        hdb_resale_data.loc[i, 'longitude'] = best_longitude
    else:
        print(f"Error fetching data for {search_val}: {response.status_code}")

    if (i + 1) % 20000 == 0:
        print(f"Processed {i + 1} records...")

print("Finished processing all records.")

hdb_resale_data.to_csv("data/processed/hdb_resale_data_with_coordinates.csv", index=False)
'''

'\nhdb_resale_data[\'latitude\'] = None\nhdb_resale_data[\'longitude\'] = None\n\nfor i, row in hdb_resale_data.iterrows():\n    if pd.notna(row[\'latitude\']) and pd.notna(row[\'longitude\']):\n        continue\n\n    search_val = row[\'block\'] + " " + row[\'street_name\']\n    url = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={search_val}&returnGeom=Y&getAddrDetails=Y&pageNum=1"\n    response = requests.get(url)\n\n    if response.status_code == 200:\n        best_score = 0\n        best_latitude = 0\n        best_longitude = 0\n\n        for result in response.json()[\'results\']:\n            score = difflib.SequenceMatcher(None, result[\'ADDRESS\'], search_val).ratio()\n            if  score > best_score:\n                best_score = score\n                best_latitude = result[\'LATITUDE\']\n                best_longitude = result[\'LONGITUDE\']\n\n        hdb_resale_data.loc[i, \'latitude\'] = best_latitude\n        hdb_resale_data.loc[i, \'longitude\'] = 

### 2. Extract and Clean MRT Station Coordinates

In [ ]:
# ------------------------------------------------------------------------------
# Downloaded MRT location data from https://data.gov.sg/datasets/d_b39d3a0871985372d7e1637193335da5/view
# This code used once to get MRT coordinate from LTAMRTStationExitGEOJSON, remove LRTs, and save coordinates to CSV
# ------------------------------------------------------------------------------

import geopandas as gpd
from bs4 import BeautifulSoup

# Load GeoJSON into a GeoDataFrame
gdf = gpd.read_file('data/raw/LTAMRTStationExitGEOJSON.geojson')

# Extract x, y, z coordinates into separate columns
gdf['longitude'] = gdf.geometry.x
gdf['latitude'] = gdf.geometry.y

def extract_station_name(html_str):
    soup = BeautifulSoup(html_str, 'html.parser')
    rows = soup.find_all('tr')
    for row in rows:
        th = row.find('th')
        td = row.find('td')
        if th and th.text.strip() == 'STATION_NA':
            return td.text.strip()
    return None

# Apply to your dataframe column 'Description'
gdf['station_name'] = gdf['Description'].apply(extract_station_name)

# Remove LRT stations (filter rows where station name contains 'LRT')
gdf = gdf[~gdf['station_name'].str.contains('LRT', na=False)]

# Group by unique station name to get average coordinates
unique_mrt_stations = (
    gdf.groupby('station_name')
    .agg({
        'latitude': 'mean',
        'longitude': 'mean'
    })
    .reset_index()
)

# Save to CSV
unique_mrt_stations.to_csv("data/processed/mrt_station_with_coordinates.csv", index=False)

### 3. Compute MRT Distance and Nearby Station Count for Each HDB unit

In [16]:
hdb_resale_data_with_coordinates = pd.read_csv("data/processed/hdb_resale_data_with_coordinates.csv")
mrt_location_data = pd.read_csv("data/processed/mrt_station_with_coordinates.csv")

In [69]:
from sklearn.neighbors import BallTree

# Convert coordinates to radians
mrt_location_in_rad = np.radians(mrt_location_data[['latitude', 'longitude']].values)
hdb_resale_data_in_rad = np.radians(hdb_resale_data_with_coordinates[['latitude', 'longitude']].values)

# Create BallTree with MRT locations using Haversine metric
tree = BallTree(mrt_location_in_rad, metric='haversine')

# Define walkable distance
walkable_distance_km = 1
adjusted_straight_line_km = walkable_distance_km / 1.4  # Adjust for real-world walking paths (non-linear)
radius_radian = adjusted_straight_line_km / 6371.0  # Convert straight-line distance to radians for Haversine

# Get num of MRT in walkable distance
indices_within_radius = tree.query_radius(hdb_resale_data_in_rad, r=radius_radian)
hdb_resale_data['num_mrts_within_1km'] = [len(indices) for indices in indices_within_radius]

# Distance to nearest MRT
distances_rad, nearest_indices = tree.query(hdb_resale_data_in_rad, k=1)
hdb_resale_data['min_dist_to_mrt_km'] = distances_rad.flatten() * 6371.0

# Save to CSV for backup
hdb_resale_data.to_csv("data/processed/hdb_resale_data_final.csv", index=False)


In [68]:
hdb_resale_data.head()

,_id,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,num_mrts_within_1km,min_dist_to_mrt_km
0,1,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0,1,0.352915
1,2,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0,0,0.816023
2,3,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0,1,0.229604
3,4,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0,1,0.664472
4,5,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0,0,0.923402
